## Jokes

For the first example, let's fetch the HTML code for the web page found at http://pun.me/pages/funny-jokes.php. There is a list of 35 jokes on this page. If you don't already have Google Chrome installed, take a moment to do that.

Open the URL in Chrome and then open Chrome Developer Tools. To do this, click on the More Options icon in the top right hand corner of your Chrome window, then click *More Tools* then *Developer Tools*.

![Open Dev Tools](assets/scraping-1.jpg)

In the developer tools window, select the *Elements* tab, or select `Ctrl + Shift + I` Windows / `Command + Shift + I` Mac. In this tab, the HTML code for the web page is displayed. As you move your mouse pointer over the code, the relevant parts of the webpage will be highlighted. See if you can find the HTML code associated with the jokes themselves.

![HTML Code](assets/scraping-2.png)

You can also access the HTML related to the jokes by hovering over one with your mouse. Right-click and select "Inspect" and you will arrive at the relevant HTML code. 

![right-click-element](assets/inspect-element-right-click.gif)


There is a lot going on in the HTML. As you drill down into the code, you will find a section of code that starts like this:

```html
<div style="float:left;width:100%;">
    <div class="content">
        <p>
            Our most-liked jokes which are genuinely funny - this list of jokes has been hand selected and contain a variety of clever, clean and silly jokes so be prepared to laugh.
        </p>
        <ol>
         <li>Today at the bank, an old lady asked me to help check her balance. So I pushed her over.</li>
        <li>I bought some shoes from a drug dealer. I don't know what he laced them with, but I've been tripping all day.</li>
```


As you move your mouse pointer over that section of code the list of jokes itself is highlighted. In order to be able to parse the HTML code to extract the data, you will need to understand a little about the structure of HTML. If you are already familiar with HTML then you are all set, but if you are new to the language then there are a few things to be aware of.

HTML is made up of elements. An element is constructed from tags. A tag looks like this: `<tagname>`. In the code above, you can see that there are `<div>` tags, `<p>` tags, an `<ol>` tag and some `<li>` tags. Each tag has a corresponding closing tag, which looks just like the tag but with a slash. So the `<li>` tag has a corresponding closing `</li>` tag.

An `li` element is made up of the opening and closing `li` tags with some content between them like this:

```html
<li>Today at the bank, an old lady asked me to help check her balance. So I pushed her over.</li>
```

The element simply tells the browser what to display and maybe some information about the content. For instance, `<p>` is for a paragraph of text, `<ol>` is for ordered lists (that is, numbered lists), and `<li>` is a list item. Notice how the `<ol>` element contains a number of `<li>` elements. Notice also that each `<li>` element corresponds to a single numbered item in the list.

There are a great many other elements on the page, but they are for headings, menus, and navigations that we are not interested in right now. You can typically use the Chrome Developer tools to drill down to the most relevant elements on the page.

This example is simple enough that the URL does not need any special attention.


## Fetch the Code

The next step is to fetch the HTML code from the website using the Python `requests` library. This is the same library that you used to request data from an API earlier, except in this case we are not fetching JSON data. Instead, we can use the `text` property to get the HTML code.

In [ ]:
import requests
url = ' http://pun.me/pages/funny-jokes.php'
response = requests.get(url)

# make sure we got a valid response
if(response.ok):
  # get the full data from the response
  data = response.text
  print(data)

In [ ]:
import requests 
url = 'https://pun.me/pages/funny-jokes.php'
response = requests.get(url)

#make sure we get a valid response 

if response.ok:
    data = response.text
    print(data)

## Beautiful Soup

Parsing the HTML code that makes up a web page can be quite difficult especially as there is no guarantee that the code is formatted correctly, or consistently with any standard. Web pages are notoriously broken. Your web browser does a heroic job of rendering web pages even when they are broken, so even if you visit a website and it looks fine, that does not mean that the code is actually fine. In addition, to build a polite scraper as defined above requires even more complexity in your code. There are several good libraries that can help you navigate through HTML. We will use [BeatifulSoup](https://www.crummy.com/software/BeautifulSoup/) to simplify the parsing.


To use BeautifulSoup we will:

* import the library
* create an object with the response text 
 
 
BeautifulSoup supports several different HTML parsers. That is, there are different ways that a program may read and understand an HTML page. when creating the object out of the HTML we need to tell BeautifulSoup which parser to use. We will use the default parser to avoid having to install additional dependencies.

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(data, 'html.parser')
#soup == objecttype BeautifulSoup-->> object represents the entire HTML document. 

The variable `soup` in this code now contains an object of type *BeautfulSoup*. This object represents the entire HTML document. We can see what it contains with the `prettify()` method.

In [ ]:
print(soup.prettify())

## Navigating the Document

Now that we have the document parsed, we need to be able to target just the elements of the document that we need. The page contains many different elements, most of which we can ignore. Let's try to find the ones we want. Go back to Chrome Dev tools and find the list of jokes in the HTML. You will notice that each joke is in an `<li>` and together all the `<li>`s  are in an `<ol>` element. Let's try to access that element.

To get a list of a certain type of element we can use the `find_all()` method. This is probably the most common method for navigating through a document looking for specific tags.

In [ ]:
soup.find_all('ol')

In [ ]:
soup.find_all('ol') # make a list of all ol elements

That did the trick. When we are only expecting a single element we can use the `find()` method instead. Now that we have the list, let's try to get only the `<li>` elements from the list.

In [ ]:
# get the list 

list = soup.find('ol')
items = list.find_all('li')
print(items )

That produces a list of the `<li>` tags. Finally, we want to extract just the text from within those tags.

In [ ]:
jokes = [joke.get_text() for joke in items]
print(jokes)

In [ ]:
jokes = [joke.get_text() for joke in items]
print(jokes)

So that did the trick. One problem that we may face is if there were multuple `<ol>` elements on the page. In that case, we would want to be as specific as possible. In HTML each element can be given a *class* attribute and an *id* attribute. More than one element may have the same class, but ids are supposed to be unique.

A quick examination of the HTML shows that the `<ol>` element has neither a class nor an id. But, the `<div>` element that encloses it has the class *content*. We could select the `<div>` with class *content* then select the `<ol>` from within. Remember that even though this step isn't really necessary for this particular page, we want to make a robust scraper that will work even if the web site owner adds more lists to the same page.

As a quick illustration of this lets do exactly what was done above but with a different route to find the jokes.

In [ ]:
#this is what we want from div class content
# soup.find_all('div')

#</div>, <div style="float:left;width:100%;">
#  <div class="content">

In [ ]:
div = soup.find('div', class_= 'content')
list = div.find('ol')
items = list.find_all('li')
jokes = [joke.get_text() for joke in items]
print(jokes)

In [ ]:
# this gets a list of all divs on the page
# soup.find_all('div')

# get just the divs with class *content*
div = soup.find('div', class_='content')
list = div.find('ol')
items = list.find_all('li')
jokes = [joke.get_text() for joke in items]
print(jokes)

## Developer Jobs

Imagine that you've been tasked with creating a presentation on current trends in developer jobs. Your audience wants to understand things like which technologies are most in demand and what perks employers are using to attract talent. Your preliminary research reveals that the StackOverflow Jobs would be a good data source.

This website lists jobs that are targeted at developers and people in the software development community. On the page, there are a number of jobs listed, along with ads and other information. But this site uses pagination to display a long list of jobs one page at a time. To see the second page you can click the Next button at the bottom of the list. To scrape useful information from this website we will need to follow that link.

Let’s use the Chrome Developer Tools to examine the page a bit.

![Jobs](assets/scraping-3.jpg)

Drilling into the page structure you will find that there is a `<div>` element with class `listResults` that contains all the job listings. Each job listing has an HTML structure similar to this:

```html
<div data-jobid="240993" class="-item -job p24 pl48 bb ps-relative bc-black-2 js-dismiss-overlay-container   can-apply ">
  <div class="dismiss-overlay ps-absolute ta-center p16 t0 r0 b0 l0 grid ai-center jc-center o90 bg-black-050 fs-body3">
    <p class="mb0">
      Okay, you won’t see this job anymore. <a href="#" class="js-undismiss-job" data-id="240993">Undo</a>
    </p>
  </div>
  <div class="dismiss-trigger js-dismiss-job ps-absolute r12 fc-black-500 c-pointer" data-id="240993" data-referrer="JobSearch" title="Dismiss job"><svg aria-hidden="true" class="svg-icon iconClearSm" width="14" height="14" viewBox="0 0 14 14"><path d="M12 3.41L10.59 2 7 5.59 3.41 2 2 3.41 5.59 7 2 10.59 3.41 12 7 8.41 10.59 12 12 10.59 8.41 7z"></path></svg></div>
  <div class="-job-summary">
    <div class="-title">
      <span data-href="https://stackoverflow.com/users/signup?returnUrl=%2fjobs%2fset-favorite%2f240993%3freturnUrl%3d%252Fjobs%253Fmed%253Dsite-ui%2526ref%253Djobs-tab%26referrer%3dJobSearch%26sec%3dFalse&amp;ssrc=jobs" data-jobid="240993" class="fav-toggle ps-absolute l16 c-pointer js-fav-toggle " title="Click to add this job to your favorites." data-ga-label="Toptal | Front-End Developer @ PUB Team | 240993" )="">
        <svg aria-hidden="true" class="svg-icon iconStar" width="18" height="18" viewBox="0 0 18 18"><path d="M9 12.65l-5.29 3.63 1.82-6.15L.44 6.22l6.42-.17L9 0l2.14 6.05 6.42.17-5.1 3.9 1.83 6.16z"></path></svg>
      </span>
      <h2 class="fs-subheading job-details__spaced mb4">
        <a href="/jobs/240993/front-end-developer-pub-team-toptal?a=1iOW90N7AGru&amp;so=i&amp;pg=1&amp;offset=24&amp;total=965" title="Front-End Developer @ PUB Team" class="s-link s-link__visited">Front-End Developer @ PUB Team</a>        
      </h2>
      <span class="ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t24">5d ago</span>
    </div>
    <div class="fc-black-700 fs-body2 -company">
        <span>Toptal</span>
        <span class="fc-black-500">
             - No office location        
        </span>
    </div>
    <div class="mt2 -perks">
      <span class="-remote pr16">Remote</span>
    </div>
    <div class="mt12 -tags">
      <a href="/jobs/developer-jobs-using-javascript" class="post-tag job-link no-tag-menu">javascript</a>
      <a href="/jobs/developer-jobs-using-node.js" class="post-tag job-link no-tag-menu">node.js</a>
      <a href="/jobs/developer-jobs-using-css" class="post-tag job-link no-tag-menu">css</a>
      <a href="/jobs/developer-jobs-using-html" class="post-tag job-link no-tag-menu">html</a>
      <a href="/jobs/developer-jobs-using-reactjs" class="post-tag job-link no-tag-menu">reactjs</a>
    </div>
  </div>
</div>
```

So there is still a lot of extraneous information in there. We're only really interested in the job title, the company, the list of technologies, and the perks. Let's use BeautifulSoup to try to select the parts that we need.

### Medium Article Example

In [ ]:
https://medium.com/@nveenverma/web-scraping-tutorial-project-scraping-stack-overflow-e28bb139fc3b

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.style as style 
from matplotlib import pyplot

%matplotlib inline

# soup = BeautifulSoup(data, 'html.parser')
style.use('fivethirtyeight') 

In [ ]:
url = 'https://stackoverflow.com/tags'

response = requests.get(url)

In [ ]:
response.status_code

In [ ]:
soup = bs(response.content, 'html.parser')

body = soup.find('body')

type(body)

In [ ]:
lang_tags = body.find_all('a', class_ = 'post-tag')
languages = [i.text for i in lang_tags]
languages[:10]

In [ ]:
tag_counts = body.find_all('span', class_ = 'item-multiplier-count')
noTags= [int(i.text) for i in tag_counts]
listed = list(zip(languages, noTags))
listed

# join the two lists in a dataframe 


In [ ]:
# Function to check, if there is any error in length of the extracted bs4 object

def error_checking(list_name, length):
    if (len(list_name) != length):
        print('Error in {} parsing, length not equal to {}!!!'.format(list_name, length))
        return -1
    else:
        pass
    
def get_top_languages(url):
    response = requests.get(url)
    
    #Parsing html data using Beautiful Soup
    soup = bs(response.content, 'html.parser')
    body = soup.find('body')
    
    #Extracting top languages 
    lang_tags = body.find_all('a', class_ = 'post-tag')
    error_checking(lang_tags, 36)
    languages = [i.text for i in lang_tags]
    
    #Extracting tag Counts
    tag_counts = body.find_all('span', class_ = 'item-multiplier-count')
    error_checking(tag_counts, 36)
    no_of_tags = [int(i.text) for i in tag_counts]
    
    # Join 2 lists together
    
    df = pd.DataFrame({'Languages':languages, 'Tag Count': no_of_tags})


    return df



In [ ]:
URL1 = 'https://stackoverflow.com/tags'

df = get_top_languages(url)
df.head()

In [ ]:
plt.figure(figsize=(8,3))
plt.bar(height=df['Tag Count'][:10], x = df['Languages'][:10])
plt.xticks(rotation = 90)
plt.xlabel('Languages')
plt.savefig('lang_vs_tag_counts.png', bbox_inches = 'tight')
plt.show()

In [ ]:
Download Webpage from stackoverflow
Parse the document content into BeautifulSoup
Extract Top Questions
Extract their respective Summary
Extract their respective Tags
Extract their respective no. of votes, answers and views
Put all code togther and join the lists
Plot Data

In [ ]:
p a — finds all a tags inside of a p tag.
# Selecting a tags inside p tag
soup.select('p a')
div.outer-text - finds all div tags with a class of outer-text.
div#first - finds all div tags with an id of first.
body p.outer-text - finds any p tags with a class of outer-text inside of a body tag.

In [ ]:
url = 'https://stackoverflow.com/questions?sort=votes&pagesize=50'
response1 = requests.get(url)
response1.status_code

In [ ]:
#printing html data using Beautiful Soup

soup1 = bs(response1.content , 'html.parser')

#body1
body1 = soup1.select_one('body')

#
type(body1)

## Question Hyperlink

In [ ]:
# We can see that the question is inside a tag, which has a class of question-hyperlink
# Taking cue from our previous Goal, we can use this class along with a tag, to extract all the question links in a list. 
# However, there are more question hyperlinks in sidebar which will also be extracted in this case. 
#To avoid this scenario, we can combine a tag, question-hyperlink class with their parent h3 tag. This will give us exactly 50 Tags.
question_links = body1.select("h3 a.question-hyperlink")

questions = [i.text for i in question_links]
questions[:2]

# q_tags = body.find_all('a', class_ = 'post-tag')
# questions = [i.text for i in q_tags]
# questions[:10]

## Extract Summary

In [ ]:
summary_divs= body1.select("div.excerpt")
summary_divs[0]
summaries = [i.text.strip() for i in summary_divs]
# summaries[0]

## Extract Tags

In [ ]:
Extracting tags per question is the most complex task in this post. 
Here, we cannot find unique class or id for each tag, and there are multiple tags per question that we need to store

In [ ]:
# Step 1 using 3rrd child overall and second div child of above extracted object. Use nth-of-type() method 
tag_divs = body1.select("div.summary > div:nth-of-type(2)")
tag_divs[0]


In [ ]:
# Step 2  Now, we can use list comprehension to extract a tags in a list, grouped per question.

a_tags_list = [i.select('a') for i in tag_divs]

#Printing first question's a tags
a_tags_list[0]

In [ ]:
# Step 3 Now we will run a for loop for going through each question and use list comprehension inside it, to extract the tags names.

tags = []

for a_group in a_tags_list:
    tags.append([a.text for a in a_group])
    
tags[0]

In [ ]:
# Number of Votes...>>>>> Theycan be found by using span tag along with vote-count-post class and nested strong tags (refer Figure 2.).

vote_spans = body1.select("span.vote-count-post strong")
print(vote_spans[:2])

In [ ]:
no_of_votes = [int(i.text) for i in vote_spans]
no_of_votes[:5]

In [ ]:
# Number of Answers They can be found by using div tag along with status class and nested strong tags. 
#Here, we don't use answered-accepted because its not common among all questions,
# few of them (whose answer are not accepted) have the class - answered (refer Figure 2.)

answer_divs = body1.select("div.status strong")
answer_divs[:2]

In [ ]:
no_of_answers = [int(i.text) for i in answer_divs]
no_of_answers[:5]

In [ ]:
# No. of Views
answer_views = body1.select("div.supernova")
answer_views[0]
# now we need to clean string and turn into an integer format 



In [ ]:
no_of_views = [i['title'] for i in answer_views]
no_of_views = [i[:-6].replace(',','') for i in no_of_views]
no_of_views = [int(i) for i in no_of_views]
no_of_views[:5]


### Assembling a DataFrame of StackOverflow All Questions Data. 

In [ ]:
def get_top_questions(url, question_count):
    # WARNING: Only enter one of these 3 values [15, 30, 50].
    # Since, stackoverflow, doesn't display any other size questions list
    url = url + "?sort=votes&pagesize={}".format(question_count)
    
    #Using requests module for downloading webpage content
    response = requests.get(url)
    
    # Parsing html data
    soup = bs(response.content, 'html.parser')
    body = soup.find('body')
    
    # Extracting Top Questions
    question_links = body1.select("h3 a.question-hyperlink")
    error_checking(question_links, question_count)
    questions = [i.text for i in question_links]
    
    #Extracting Tags
    tags_div = body1.select("div.summary > div:nth-of-type(2)")
    
    error_checking(tags_div, question_count)
    a_tags_list = [i.select('a') for i in tags_div]
    
    tags = []
    
    for a_group in a_tags_list:
        tags.append([a.text for a in a_group])
        
    # Extracting Number of Votes
    vote_spans = body1.select("span.vote-count-post strong")
    error_checking(vote_spans,question_count)
    no_of_votes = [int(i.text) for i in vote_spans]
    
    
    #Extracting Number of answers 
    answer_divs = body1.select("div.status strong")
    error_checking(answer_divs, question_count)
    no_of_answers = [int(i.text) for i in answer_divs]
    
    # Extracting Number of Views
    
    div_views = body1.select("div.supernova")
    error_checking(div_views, question_count)
    
    no_of_views = [i['title'] for i in answer_views]
    no_of_views = [i[:-6].replace(',','') for i in no_of_views]
    no_of_views = [int(i) for i in no_of_views] 
    
    # Putting them all together 
    
    df = pd.DataFrame({'question':questions, 'summary':summaries,
                      'tags':tags, 'no_of_votes': no_of_votes,
                      'no_of_answers': no_of_answers, 'no_of_views': no_of_views})
    
    
    
    
    
    return df

In [ ]:
URL2 = 'https://stackoverflow.com/questions'

df1 = get_top_questions(URL2, 50)
df1.head()

In [ ]:
f, ax = plt.subplots(3,1, figsize = (12,8))
ax[0].bar(df1.index, df1.no_of_votes)
ax[0].set_ylabel('No of Votes')

ax[1].bar(df1.index, df1.no_of_views)
ax[1].set_ylabel('No of Views')

ax[2].bar(df1.index, df1.no_of_answers)
ax[2].set_ylabel('No of Answers')

plt.savefig('votes_vs_views_vs_answers.png', bbox_inches ='tight')

plt.show()

In [ ]:
url = 'https://stackoverflow.com/jobs'
response = requests.get(url)

if (response.ok):
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    
    # find all the elements with class *-job-summary*
    summary = soup.select('.-job-summary > .-company > span')
    print(summary)

In [130]:
import requests 
from bs4 import BeautifulSoup as bs

In [132]:
url = 'https://stackoverflow.com/jobs'
response = requests.get(url)
response.status_code

200

In [143]:
soup = bs(response.text, 'html.parser')
jobs_list = soup.find_all('a', class_= 's-link stretched-link')

In [144]:
jobs = [job.text for job in jobs_list]

jobs

['Senior Software Engineer',
 'Sitecore Developer (C#, JavaScript, HTML, CSS)',
 'Java Developer with Kafka',
 'Lead Software Engineer - C#, Java, Node.js, JavaScript',
 'Senior Software Engineer',
 'Lead or Senior Software Engineer',
 'Lead or Senior Software Engineer',
 'Full Stack Software Engineer',
 'Full Stack Developer',
 'Frontend Entwickler Angular (m/w/d)',
 'Senior Software Engineer',
 'Senior Software Engineer (Frome)',
 'Full Stack Software Engineer',
 'Full-Stack Web Developer',
 'Arquitecto /a de Software',
 'FULL STACK DEVELOPER (M/W/D)',
 'Full Stack Application Developer',
 'Java Developer',
 'Full Stack Software Engineer',
 'Software Engineer Lead',
 'Software Engineer (React / Node.js)',
 'Python Developer (Onsite in Paris, or Remote in France, UK, Germany)',
 'Web Developer Full Stack (H/F)',
 'Maquetador / Programador Junior',
 'Full Stack Software Engineer (f/m/d)']

In [ ]:
url = 'https://stackoverflow.com/jobs'
response = requests.get(url)

# make sure we got a valid response
if(response.ok):
  # get the full data from the response
  data = response.text
  soup = BeautifulSoup(data, 'html.parser')
  
  # find all elements with class *-job-summary*
  summary =soup.find_all(class_='-job-summary')
  #print(summary)

Not bad! That gave us a list of the divs containing job info. But, that is still a lot of extraneous HTML. We could narrow it down further by just geting the titles.

In [ ]:
titles = soup.find_all(class_='title')
print(titles)

Better, but the actual title text that we want is in an `<a>` tag inside an `<h2>` tag. We can continue navigating down the document or we can use another technique: [**CSS Selectors**](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Selectors). CSS is the language used to apply styles to web pages. CSS selectors are used to select specific elements on the page so that styles may be applied to just those elements. But the syntax is so powerful that we can use the same selector syntax in BeautifulSoup. 

To select all elements with a particular class use the `.classname` selector.

In [ ]:
titles = soup.select('.-title')
print(titles)

This gives us the same output as earlier. So let's extend this a bit. To select only the `<h2>` elements in the divs with class *-title*, we can use the **child** selector. The `>` symbol is used to specify a child element. A child element is an element contained within another element.

In [ ]:
title_headings = soup.select('.-title > h2')
print(title_headings)

Okay. That's better, but it'd be even better if we selected only the `<a>` tags `<h2>`s.

In [ ]:
title_links = soup.select('.-title > h2 > a')
print(title_links)

Now, we can use `get_text()` as we did above to extract the text content of the links.

In [ ]:
jobs = [job.get_text() for job in title_links]
print(jobs)

How would we get a list of the companies that posted jobs on this page? Take a few minutes and try to construct the CSS selector expression that will return this information.

It may be obvious from the HTML that the company information is inside the job summary in a `<div>` with class `-company`. If you select the `<span>`s content, you will get the company information.

In [ ]:
company_spans = soup.select('.-job-summary > .-company > span')
companies = [company.get_text() for company in company_spans]
print(companies)

We are getting the information that we want, but there's extra text here that we do not want. Look carefully at the div with class *-company* and notice that it actually contain multiple spans. The company name is usually the first one. We can use the `nth-of-type()` selector to, say, only grab me the first match.

In [ ]:
company_spans = soup.select('.-job-summary > .-company > span:nth-of-type(1)')
companies = [company.get_text() for company in company_spans]
print(companies)

Getting the tags for each job is much the same. Let's try to put together some code that will get all the information about the job that we need and put it into a dictionary object.


In [ ]:
# iterate over all jobs
raw_jobs = soup.select('.-job-summary')
jobs = []
for job in raw_jobs:
  title = job.select_one('.-title > h2 > a').get_text() # extract the title
  company = job.select_one('.-company > span:nth-of-type(1)').get_text().strip() # extract the company
  tags = [tag.get_text() for tag in job.select('.-tags a')] # extract a list of tags
  job = {'title': title, 'company': company, 'tags': tags} # construct a dictionary
  jobs.append(job) # add dictionary to list
print(jobs)  

## Following a link
Now that we can scrape the list of jobs from the first page, what about the other pages? Let's look at the HTML code for the next button.

```html
<a href="/jobs?sort=i&pg=2" title="page 2 of 39" class="prev-next job-link test-pagination-next">
  <span class="text">next</span>
  <i class="material-icons">chevron_right</i>
</a>
```

The next button is made up of a clickable link styled to look like a button. The `href` attribute of the link contains the URL for the next page. The URL of this button is `/jobs?sort=i&pg=2`. Notice that this is only part of the full URL for that page. A URL is made up of the domain name and path and optional query string. Since we are on the `https://stackoverflow.com/jobs` website, the browser knows that if we click this link we want to remain on the same website and just visit a different path. So the actual request will be sent to `https://stackoverflow.com/jobs?sort=i&pg=2`. The bit after the `?` is called the query string, and it is one way to send data to the server describing what you are requesting. It appears that this link will accept a parameter named `pg` representing the page number. If you click on the next button several times you will notice that the bit at the end of the URL that says `pg=2` changes to `pg=3` then `pg=4` and so on. 

That means that we can modify the URL by incrementing the page number to request more jobs. We can continue to do so until there are no more pages. 

To set the query string on the request URL, we can use the `params` parameter on the get method. For example,

```python
page_number = 1
query = {'sort':'i', 'pg': page_number}
url = 'https://stackoverflow.com/jobs'
response = requests.get(url, params=query)
```

We could then update the `page_number` variable and repeat this call.

This raises a few questions:

 - How do we know when there are no more pages?
 - Of we rapidly send many requests for many pages to the server, will we overwhelm the server and possibly get ourselves blocked?
 
There may be several answers to the first question. If you go to the website and visit the last page of results you will see that the next button is not displayed on that page. So the absence of the next button could be the condition on which we stop the processing. There may be others that you can readily spot but we will go with that option for now.

The second one does not rely on the HTML structure of the page, but rather on the way we code up the solution. The above code would process a page of jobs and create a list of dictionaries from that page. To repeat that we need some sort of loop that will iterate over all pages and for each page perform something similar to the code that we already wrote. When we get to the last page we stop the loop and at that point we will have a list of all jobs on the website. The problem here is that our program will process each page much more rapidly than a human browsing the website would. If we're not careful, this could appear to be a denial of service attack against the website, and since most web servers can identify such attacks and take steps to protect themselves, we want to avoid this behavior. 

One solution is to deliberately slow down the rate of requests that we make. After making one request, we can wait a few seconds before making another one. If we wait 1 second between requests and we need to request 500 pages, then that is at least 500 seconds for our program to run. That is not so bad since hopefully scraping the data is a one time affair. There are some libraries that help to moderate your request rate if you are buiding a more sofisticated scraper. For our simple scraper, we can use the Python's `sleep()` method to pause execution for a short time.

## Some other considerations

Before we jump into code, there are a few other considerations that we should make since we are requesting multiple pages. We should clearly identify ourselves to the web server. That is, when your browser makes a request to a web server it sends along a header named **User-Agent** with a value that identifies the browser itself. For instance, in Chrome the value looks like this:

```
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36
```

And in Firefox the value looks like this:

```
Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:63.0) Gecko/20100101 Firefox/63.0
```

This way the web server can track how many visitors use a particular browser. We can set a value that identifies us and at least provide a contact. If the website owners wish, they may contact us to get some clarity on why we are scraping data from their website and maybe negotiate a better experience for themselves as well as us. We could provide a value like:

```
'jobscraper - school project (yourname@gmail.com)'
```

For example, when making the request we could set the header like this:

```
url = '...'
headers = {'user-agent': 'jobscraper - school project (yourname@gmail.com)'}
response = requests.get(url, headers=headers)
```

Another consideration is monitoring the program as it runs. We are creating a program that may take 10 to 20 minutes or even more to run. It would be important to "see" what is happening as the program runs. This helps us to follow the progress, ensure the  program isn't hung up, and to detect problems early. To this end, we can simply print the number of requests that have been made so far, the frequency of the requests, the number of items found so far and any errors encountered.

Putting all of this together can be a bit daunting, but careful step-by-step thought will make it possible. A complete working program is given below. Before you run it, ensure you set your email address in the header. Also, notice the variable `MAX_REQUESTS` which is used to set an upper limit on the number of requests made. You may adjust this value as you wish.

In [ ]:
# import all dependencies at the top
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn
from bs4 import BeautifulSoup
import requests


# define a function to process the page
def process_page(soup, jobs):  
  
  # find all elements with class *-job-summary*
  raw_jobs = soup.select('.-job-summary')

  # same as above, extract the info we need
  for job in raw_jobs:
    title = job.select_one('.-title > h2 > a').get_text() # extract the title
    company = job.select_one('.-company > span:nth-of-type(1)').get_text().strip() # extract the company
    tags = [tag.get_text() for tag in job.select('.-tags a')] # extract a list of tags
    job = {'title': title, 'company': company, 'tags': tags} # construct a dictionary
    jobs.append(job) # add dictionary to list

    
# prepare for the monitoring logic
start_time = time() # note the system time when the program starts
request_count = 0 # track the number of requests made

# create a list to store the data in
jobs = []

# variables to handle the request loop
has_next_page = True
MAX_REQUESTS = 100 # do not request more than 100 pages
page_number = 1
query = {'sort':'i', 'pg': page_number}
url = 'https://stackoverflow.com/jobs'
headers = {'user-agent': 'jobscraper - school project (myeamail@gmail.com)'}

while has_next_page and request_count <= MAX_REQUESTS:
  # keep the output clear
  clear_output(wait = True)
  
  # make an initial request
  response = requests.get(url, params=query, headers=headers)

  # make sure we got a valid response
  if(response.ok):
    # get the full data from the response
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    process_page(soup, jobs)

    # check for the next page
    # look for the presence of element with class *test-pagination-next*
    next_button = soup.select('.test-pagination-next')
    has_next_page = len(next_button) > 0
    
  else:
    # display a warning if there are any problems
    warn('Request #: {}, Failed with status code: {}'.format(request_count, response.status_code))
  
  request_count += 1
  
  # go to sleep for a bit
  # we use a random number between 1 and 5 so
  # We can wait as long as 5 seconds to make a second request
  
  sleep(randint(1,3))
  
  # output some logs for monitoring
  elapsed_time = time() - start_time
  print('Requests: {}, Frequency: {} requests/s, {} jobs processed.'.format(request_count, request_count/elapsed_time, len(jobs)))
  
  # prepare for next iteration
  page_number += 1
      
print('Sraping complete')
print('Requests: {}, Frequency: {} requests/s, {} jobs processed.'.format(request_count, request_count/elapsed_time, len(jobs)))

In [ ]:
# print the first five jobs
jobs[0:5]

## Save to a file
Getting the data from the server is only the first part of the problem. The next step is to persist the data in some format that makes it available for analysis later. A CSV file is a fairly common format and well supported in many environments. The `csv` Python library provides the tools to read and write csv files in Python. 

## Note about Colab

Since this is a hosted service writing a file to the filesystem is not quite as straightforward as if this code was running on your local machine. We can get around this by writing to your Google Drive. In order to use Google Drive, you will need to be authenticated. Luckily, Google provides a library named *PyDrive* that makes this simple.

First, ensure that PyDrive is installed in the environment. This command only needs to be executed once in a notebook.


In [ ]:
# install this library
!pip install -U -q PyDrive

To authenticate with Google, do the following. Note that you will be prompted to login with your Google credentials and then you will be given a code to enter. When you enter that code, this notebook will have permission to write files to your Google Drive.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Next, we can create a csv string of our jobs and write it to Google Drive with a file  name of "jobs.csv"

In [ ]:
import csv
import io

# Create an output stream
output = io.StringIO()

# these are the names of the properties in the dictionary
fieldnames = ['title', 'company', 'tags']

# create a writer object, it can write dictionaries to the output stream
writer = csv.DictWriter(output, fieldnames=fieldnames)

# write all the headings 
writer.writeheader()

# iterate the jobs and write each one
for job in jobs:
  writer.writerow(job)


# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'jobs.csv'})
uploaded.SetContentString(output.getvalue())
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))  
      

Visit your Google Drive and find the file named "jobs.csv", and verify that it contains a list of jobs that was scraped from the StackOverflow jobs site.